# Exploration des données

## Initialisation

 ### Installation des librairies

In [1]:
# Mettre a jour l'environnement virtuel et les requirements.txt
%pip install -r ../requirements.txt 
%pip freeze > ../requirements.txt 

INFO: pip is looking at multiple versions of tensorflow to determine which version is compatible with other requirements. This could take a while.
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
  Using cached tensorboard-2.19.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached stack_data-0.6.3-py3-none-any.whl.metadata (18 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
INFO: pip is still looking at multiple versions of tensorflow to determine which version is compatible with other requirements. This could take a while.
  Using cached rich-14.0.0-py3-none-any.whl.metadata (18 kB)
  Using cached urllib3-2.4.0-py3-none-any.whl.metadata (6.5 kB)
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, p

ERROR: Cannot install -r ../requirements.txt (line 28), -r ../requirements.txt (line 67), protobuf==6.31.1 and tensorflow==2.19.0 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


Note: you may need to restart the kernel to use updated packages.


### Importation des librairies

In [6]:
# Kaggle API
from kaggle.api.kaggle_api_extended import KaggleApi # mettre clé API kaggle dans C:\Users\<VotreNom>\.kaggle\kaggle.json

# Global
import os
import shutil

# Data
import pandas as pd
import numpy as np

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
import tensorflow as tf


## Téléchargement Data

Télécharge les fichiers même s'ils existent déjà

In [3]:
# Authentification
api = KaggleApi()
api.authenticate()

# URL du dataset à télécharger
dataset_url = 'tawsifurrahman/covid19-radiography-database'

# Chemin du dossier où le dataset sera extrait
destination_dir = os.path.abspath("../data")
dataset_folder = os.path.join(destination_dir, dataset_url.split('/')[-1])
print("Dossier de destination :", dataset_folder)

# Création du dossier de destination principal
os.makedirs(destination_dir, exist_ok=True)

# Vérification de l'existence du dossier du dataset
if os.path.exists(dataset_folder):
    print(f"Le dossier du dataset existe déjà : {dataset_folder}")
    print("Téléchargement ignoré.")
else:
    print("Téléchargement du dataset dans :", dataset_folder)
    api.dataset_download_files(
        dataset_url,
        path=dataset_folder,
        unzip=True
    )
    print("Dataset téléchargé avec succès dans :", dataset_folder)
    

Dossier de destination : c:\Users\Léna\Documents\A_Repos\DS_COVID\data\covid19-radiography-database
Le dossier du dataset existe déjà : c:\Users\Léna\Documents\A_Repos\DS_COVID\data\covid19-radiography-database
Téléchargement ignoré.


## Analyse des Data

In [4]:
path = os.path.join(dataset_folder,os.listdir(dataset_folder)[0])
os.listdir(path)

['COVID',
 'COVID.metadata.xlsx',
 'Lung_Opacity',
 'Lung_Opacity.metadata.xlsx',
 'Normal',
 'Normal.metadata.xlsx',
 'README.md.txt',
 'Viral Pneumonia',
 'Viral Pneumonia.metadata.xlsx']

In [5]:
normal_metadata = pd.read_excel(os.path.join(path, 'Normal.metadata.xlsx'), sheet_name='Sheet1')
normal_metadata.head()

,FILE NAME,FORMAT,SIZE,URL
0,NORMAL-1,PNG,256*256,https://www.kaggle.com/c/rsna-pneumonia-detect...
1,NORMAL-2,PNG,256*256,https://www.kaggle.com/c/rsna-pneumonia-detect...
2,NORMAL-3,PNG,256*256,https://www.kaggle.com/c/rsna-pneumonia-detect...
3,NORMAL-4,PNG,256*256,https://www.kaggle.com/c/rsna-pneumonia-detect...
4,NORMAL-5,PNG,256*256,https://www.kaggle.com/c/rsna-pneumonia-detect...
